In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
#MDF_COMENT
np.set_printoptions(precision=6, linewidth=120)

# Problema 2:
Considere el entramado mostrado en la figura, con una fuerza aplicada de 20 kN. Calcule los desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los elementos tienen E = 210 GPa y una sección de 10 cm2, excepto el elemento 3, que tiene una sección de 20 cm2. Los elementos 2 y 5 tienen una longitud de 8 metros y el elemento 3 de 4 metros.

In [2]:
def loc_a_glob (z):
#    z=z*np.pi/180 # pero la arctg ya te lo da en rad!
#MDF COMMENT como me costó!
    c=np.cos(z)
    s=np.sin(z)
    K=[[ c**2,    c*s ,   -c**2, -c*s] , 
       [ c*s,      s**2,   -c*s, -s**2] , 
       [-c**2,    -c*s,    c**2,  c*s], 
       [-c*s,     -s**2,    c*s,   s**2]]
#    return K
    return np.array(K)

In [3]:
N = 4                      # cantidad de nodos = cantidad de elementos -1
# MDF_COMMENT tenes un problema de unidades. te conviene usar todo en mks :
#E = np.ones(N+1)*210            # Módulo elástico de cada elemento GPa
E = np.ones(N+1)*210e9
# A = np.ones(N+1)*10             # Area de cada elementos cm2
A = 10*np.ones(N+1)*1e-4
A[2] = 20e-4
L = np.array([0, 8, 4, 0, 8]) #MDF_COMMENT *100  # Longitud de cada elemento cm
GL = 2                               # Grados de libertad

#MDF_COMMNET esta bien para este ejercicio, pero las longitudes y las dimensiones las tenes que poder 
#MDF_COMMENT definir en a partir de la matriz de nodos.

MN = [[0,0] , [L[1],L[2]] , [L[1]+L[4],0] , [L[1],0]]  # MAtriz de nodos (posiciones de los nodos) (ejercicio)
     # 0 1       2    3       4        5      6   7
MC = [ [1, 0] , [3,0] , [1,3] , [2,3], [2, 1]  ]      # Matriz de conectividad de elementos (el  elemento 0: entre nodo 0: MN[0] y nodo 1 MN[1])  ) (ejercicio)


# ff=np.array([[2,0], [4,0], [5,-20000], [6,0]]) # columnas: nodo y eje - filas: valor de fuerza conocido
ff=np.array([[2,0], [4,0], [7,-20000], [6,0]]) # columnas: nodo y eje - filas: valor de fuerza conocido
#                          ||
# no hice una matriz de datos desplazamientos pq son todos ceros, cuando el codigo funcione, lo generalizo.

In [4]:
u = np.zeros(N*GL)


K = np.zeros([N*GL,N*GL])     # Matriz rigidez GLOBAL kN/m



for i in range(len(MC)):
    dx= MN[MC[i][0]][0] - MN[MC[i][1]][0]
    dy = MN[MC[i][0]][1] - MN[MC[i][1]][1]
    dL=np.sqrt( dx ** 2 + dy ** 2 )
    k=E[i]*A[i]/dL    
    z=np.arctan2(dy,dx)          # Creo que el error esta acá o en R
#    R=k*np.array(loc_a_glob(z))  # Matriz de rigidez local rotada
    R = k*loc_a_glob(z)
    n1=MC[i][0]*2                #index nodo 1 en x del elemento i
    n2=MC[i][1]*2                #index nodo 2 en x del elemento i
    # MDF_COMMENT es interesante que te funcionan los rangos. hay que tener cuidado igual con el
    # índice extra que necesitas
    K[n1:n1+2,n1:n1+2]+=R[0:2,0:2]
    K[n1:n1+2,n2:n2+2]+=R[0:2,2:4]
    K[n2:n2+2,n1:n1+2]+=R[2:4,0:2]
    K[n2:n2+2,n2:n2+2]+=R[2:4,2:4]
#    K[n1:n1+2,n1:n1+2]+=R[0:2,0:2]
#    K[n1:n1+2,n2:n2+2]+=R[0:2,2:4]
#    K[n2:n2+2,n1:n1+2]+=R[2:4,0:2]
#    K[n2:n2+2,n2:n2+2]+=R[2:4,2:4]

claro pero eso tambien te falta, fijate que te falta la condicion de contorno sobre los gl 0 y 1!

In [5]:
#Condiciones de contorno:

kk=np.zeros((len(ff),len(ff)))   #Matriz reducida para calcular desplazamientos desconocidos
for i in range(len(ff)):
    for j in range(len(ff)):
        kk[i, j]=K[ff[i,0], ff[j,0]]
#MDF_COMMENT no necesitas reducirla de antemano, si tenes el vector de índices de vínculos : f[:,0] la podes
#MDF_COMMENT armar con np.ix:
#MDF_COMMENT uu=np.linalg.solve(kk, ff[:,1])

_comentario_

y en realidad te falta una parte de la ecuacion que daría  cuentas de cualquier desplazamiento impuesto en los nodos.
te falta el vector s (el complemento de f[:,0]

In [6]:
ff[:,0]

array([2, 4, 7, 6])

In [7]:
I = np.linspace(0,7,8).astype(int) 
s = np.delete( I , ff[:,0] )
s

array([0, 1, 3, 5])

$$ U_r = K_{r,r} ^{-1} \left[ F_r - K_{r,s} U_s \right] $$

```.python
U[r] = np.linalg.solve(
   K[ np.ix( ff[:,0], ff[:,0] ) ] ,
   ff[:,1] - K[np.ix_( f[:,0], s ) ) ].
)
```

In [8]:
uu = np.linalg.solve(K[np.ix_(ff[:,0], ff[:,0])], ff[:,1]-K[np.ix_(ff[:,0], s)].dot(u[s]) )

In [9]:
#for i in range(len(ff)):
#    u[ff[i,0]]=uu[i]
u[ff[:,0]] = uu

Las fuerzas no es necesario resolver, sino que son simplemente una multiplicacion matricial:

In [10]:
#F=np.linalg.solve(K,u)

#F, u

In [11]:
F = K.dot(u)

In [12]:
F

array([ 1.754368e-29,  3.061617e-13,  0.000000e+00,  2.000000e+04,  1.754368e-29, -3.061617e-13,  0.000000e+00,
       -2.000000e+04])

In [13]:
F[np.abs(F/np.max(F)) < 1e-12 ]  = 0

In [14]:
F

array([     0.,      0.,      0.,  20000.,      0.,      0.,      0., -20000.])

# Confirmación de suma de rigidez local en la global:

x=np.array([[1,1,1,1], [2,2,2,2],[3,3,3,3],[4,4,4,4]])
y=np.array([[5,5,5,5], [6,6,6,6],[7,7,7,7],[8,8,8,8]])
z=np.array([[9,9,9,9], [10,10,10,10],[12,12,26,26],[13,13,25,25]])

K = np.zeros([6,6])  
e=[x,y,z]
MC=[[0,1],[1,2],[2,0]]
for i in range(len(MC)): 
    K0=copy.copy(K)
    n1=MC[i][0]*2
    n2=MC[i][1]*2
    if i==0:
        kLoc=x
    elif i==1:
        kLoc=y
    else:
        kLoc=z
    K[n1:n1+2,n1:n1+2]+=kLoc[0:2,0:2]
    K[n1:n1+2,n2:n2+2]+=kLoc[0:2,2:4]
    K[n2:n2+2,n1:n1+2]+=kLoc[2:4,0:2]
    K[n2:n2+2,n2:n2+2]+=kLoc[2:4,2:4]
    print(K-K0)